# Bharath Gunasekaran 
# CMPE-258 Part 3c



In [2]:
import tensorflow as tf 
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras.layers import Layer

### Basic prediction function: Linear

In [3]:
class Linear(keras.layers.Layer):
  """y = w.x + b"""

  def __init__(self, units=3, input_dim=3):
      super(Linear, self).__init__()
      w_init = tf.random_normal_initializer()
      self.w = tf.Variable(
          initial_value=w_init(shape=(input_dim, units), dtype='float32'),
          trainable=True)
      b_init = tf.zeros_initializer()
      self.b = tf.Variable(
          initial_value=b_init(shape=(units,), dtype='float32'),
          trainable=True)

  def call(self, inputs):
      # print(inputs.shape,self.w.shape)
      return tf.matmul(inputs, self.w) + self.b

In [9]:


class Model(keras.layers.Layer):
    # Creating Layer
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = Linear(3)
        self.linear_2 = Linear(5)
        self.linear_3 = Linear(2,5)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

In [5]:
# Make non-linear data

n = 200
d = 3
x = tf.random.normal(shape=(n,d))

weights_true = tf.constant([[5, 2,1],[1,2,1]], dtype=tf.float32)
weights_true = tf.transpose(weights_true)
bias_true = tf.constant([1], dtype=tf.float32)
y_true = (x ** 2) @ weights_true + x @ weights_true + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

x: (200, 3), weights: (3, 2), bias: (1,), y: (200, 2)


In [10]:
# Instantiate a logistic loss function that expects integer targets.
loss_fn = tf.keras.losses.MeanAbsoluteError()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
model = Model()
# Open a GradientTape.
with tf.GradientTape() as tape:

  # Forward pass.
  y_pred = model(x)

  # Loss value for this batch.
  loss = loss_fn(y_true, y_pred)
    
# Get gradients of weights wrt the loss.
print(model.trainable_variables)
gradients = tape.gradient(loss, model.trainable_variables)

# Update the weights of our linear layer.
optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Logging.
print('Loss:', float(loss))

[<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.03565552,  0.03192404,  0.06414406],
       [ 0.11720373, -0.05355733,  0.04609191],
       [ 0.06519731, -0.06866423,  0.04018589]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
array([[-0.01377905, -0.02021598,  0.00801876, -0.01219335, -0.01222613],
       [ 0.00804634,  0.05291463,  0.10484491, -0.03093672,  0.09464707],
       [ 0.01791799,  0.02170271, -0.0754884 ,  0.05896416, -0.04584379]],
      dtype=float32)>, <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'Variable:0' shape=(5, 2) dtype=float32, numpy=
array([[ 0.04110245,  0.03007048],
       [ 0.01848868, -0.01557377],
       [-0.00761119, -0.0564222 ],
       [-0.03850098, -0.08844631],
       [ 0.05794913,  0.01517935]], dtype=float32)>, <tf.Variable 'Var

In [11]:
from typing import Callable

def fit(x: tf.Tensor, y: tf.Tensor, model: Callable, loss: Callable, optimizer: Callable,  num_epochs: int):
  for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
      y_pred = model(x)
      loss_value = loss(y_pred, y)
      print(f'Epoch {epoch}, loss {loss_value}')
      gradients = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, model.trainable_weights))



# Training the Model

In [12]:
model = Model()
# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.MeanAbsoluteError()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=0.09)
fit(x, y_true, model=model, loss=loss, optimizer=optimizer, num_epochs=10)
# plot_3d(x, y_true, model(x))

Epoch 0, loss 7.1576738357543945
Epoch 1, loss 7.1176838874816895
Epoch 2, loss 7.07784366607666
Epoch 3, loss 7.038644790649414
Epoch 4, loss 7.000683784484863
Epoch 5, loss 6.963467597961426
Epoch 6, loss 6.926637649536133
Epoch 7, loss 6.890141487121582
Epoch 8, loss 6.853644847869873
Epoch 9, loss 6.817147254943848


In [13]:
def plot3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [14]:
y_pred = model(x)
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(y_pred)
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot3d(X_red,y_true_red,y_pred_red)

X_red: (200, 2), y_true_red: (200, 1), y_pred_red: (200, 1)
